In [1]:
from pathlib import Path
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from data.paths.parquet_paths import FRIENDS_VISITORS

In [14]:
df = pd.read_parquet(str(FRIENDS_VISITORS))

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1105 entries, 0 to 1104
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   business_id        1105 non-null   object
 1   business_name      1105 non-null   object
 2   user_id            1105 non-null   object
 3   user_name          1105 non-null   object
 4   friends_attendees  1105 non-null   object
dtypes: object(5)
memory usage: 43.3+ KB


In [4]:
df.sample(4)

,business_id,business_name,user_id,user_name,friends_attendees
557,Er5XAY2UFGGqBHdqpeCFkA,Zenshin Asian Restaurant,JADdo9NEeO5Az9aOYbyvZA,Manda Bear,"[{'user_id': 'UYcmGbelzRa0Q6JqzLoguw', 'user_n..."
680,1d6c6Q2j2jwVzBfX_dLHlg,Cowboy Ciao,YyhoFd8QUXxPiR4uyOxcEQ,Stephanie,"[{'user_id': 'yH5H3fJrxFXuXyC6LQYacg', 'user_n..."
246,vchjO0Vs5FwW2tpFnCimqg,Planet Hollywood Towers,uERlyO-ASZag6XXouCrkhg,Laurie,"[{'user_id': 'qK5GQYyckJx_5tyqDtqrXw', 'user_n..."
600,bsrj9_hFAql3dlSf244zpg,Trump International Hotel Las Vegas,_KDforIDsQS6bIaJi0_V0g,Elysse,"[{'user_id': 'qcLHpcGmNhxrKad5E7f7eQ', 'user_n..."


In [5]:
df.query("business_id == '1HD5iUUfVJDbfEBIn9yVhw'")

,business_id,business_name,user_id,user_name,friends_attendees
243,1HD5iUUfVJDbfEBIn9yVhw,Boba Tea House,sYtSLVYOyWavDzBU1ot5vw,Geoff,"[{'user_id': 'M9rRM6Eo5YbKLKMG5QiIPA', 'user_n..."
266,1HD5iUUfVJDbfEBIn9yVhw,Boba Tea House,Mv2qM96jDdKXJNDvzANU-g,Robin,"[{'user_id': 'M9rRM6Eo5YbKLKMG5QiIPA', 'user_n..."
267,1HD5iUUfVJDbfEBIn9yVhw,Boba Tea House,iDlkZO2iILS8Jwfdy7DP9A,Judy,"[{'user_id': 'M9rRM6Eo5YbKLKMG5QiIPA', 'user_n..."
406,1HD5iUUfVJDbfEBIn9yVhw,Boba Tea House,9uE0smG2bwgkI95RPj0lPQ,Su-shien,"[{'user_id': 'sYtSLVYOyWavDzBU1ot5vw', 'user_n..."
409,1HD5iUUfVJDbfEBIn9yVhw,Boba Tea House,DSDL_v5dvLOIJ-o5sRhz4w,Kathy,"[{'user_id': 'M9rRM6Eo5YbKLKMG5QiIPA', 'user_n..."
837,1HD5iUUfVJDbfEBIn9yVhw,Boba Tea House,M9rRM6Eo5YbKLKMG5QiIPA,Aileen,"[{'user_id': 'sYtSLVYOyWavDzBU1ot5vw', 'user_n..."
983,1HD5iUUfVJDbfEBIn9yVhw,Boba Tea House,2nE0zU6y_F7gkwHi3yL6cQ,Erin,"[{'user_id': 'iDlkZO2iILS8Jwfdy7DP9A', 'user_n..."


# Networking

user_id to friend_id

business_name to user_id

In [15]:
user_friends_network_df = pd.DataFrame({"user_name": [], "friend_name": []})

def create_friends_attendees_network(src: str, tr: str, from_df, to_df) -> pd.DataFrame:
    for i, row in from_df.iterrows():
        source = row[src]
        targets = row[tr]
        for user in targets:
            to_df.loc[len(to_df)] = [source, user['user_name']]
    
    return to_df
    
user_friends_network_df = create_friends_attendees_network('user_name', 'friends_attendees', df, user_friends_network_df)

In [16]:
print(user_friends_network_df.shape)
user_friends_network_df.sample(10)

(2616, 2)


,user_name,friend_name
1914,Anne,Jessica
592,Bill,Arana
2392,Johnny,Stephanie
2122,Nydia,Samantha
933,Robert,Paul
1158,Kevin,Todd
155,Jennifer,Ginger
1178,Belle,Emily
1922,Matthew,Rodney
1255,Lindsay,Alicia


In [8]:
# len(users_network_df)-len(users_network_df.drop_duplicates())
# users_network_df.loc[users_network_df.duplicated()]
# user_friends_network_df = user_friends_network_df.drop_duplicates(subset=['user_name', 'friend_name'])

In [9]:
user_friends_network_df.shape

(2039, 2)

In [18]:
user_friends_network_df["value"] = 1
user_friends_network_df = user_friends_network_df.groupby(["user_name","friend_name"], sort=False, as_index=False).sum()

In [20]:
user_friends_network_df.head(5)

,user_name,friend_name,value
0,Aimee,Amy,1
1,Aimee,Kevin,1
2,Aimee,Naqvi,1
3,Jennifer,David,2
4,Jennifer,Naqvi,1


In [21]:
user_friends_network_graph = nx.from_pandas_edgelist(
    user_friends_network_df, source='user_name',
    target='friend_name', edge_attr='value', create_using=nx.Graph()
)

In [22]:
from pyvis.network import Network
from community.community_louvain import best_partition

node_degree = dict(user_friends_network_graph.degree)

#Setting up node size attribute
nx.set_node_attributes(user_friends_network_graph, node_degree, 'size')

In [23]:
communities = best_partition(user_friends_network_graph)
nx.set_node_attributes(user_friends_network_graph, communities, 'group')

In [24]:
com_net = Network(notebook=True, width="1800px", height="1013px", bgcolor='#222222', font_color='white')
com_net.from_nx(user_friends_network_graph)
com_net.show("graph-user-friends.html")

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
